In [1]:
 # Dependencies
import requests
import urllib
import random
import math
import pandas as pd
import xml.etree.ElementTree as ET
from config import zws_id, gkey 
from urllib.request import urlopen

In [2]:
# function to grab the exact address based on longitude and latitude
# modified from here https://gist.github.com/bradmontgomery/5397472
# their example didn't include an API key, but I added it otherwise you'd hit the rate limit easily

def reverse_geocode(latitude, longitude):
    # Did the geocoding request comes from a device with a
    # location sensor? Must be either true or false
    sensor = 'true'

    # Hit Google's reverse geocoder directly
    # NOTE: I *think* their terms state that you're supposed to
    # use google maps if you use their api for anything.
    base = "https://maps.googleapis.com/maps/api/geocode/json?"
    params = "latlng={lat},{lon}&sensor={sen}&key={key}".format(
        lat=latitude,
        lon=longitude,
        sen=sensor,
        key=gkey
    )
    url = "{base}{params}".format(base=base, params=params)
    #print(url)
    response = requests.get(url).json()
    address = response['results'][0]['formatted_address']
    return address


In [3]:
# function to generate random lat & lng within a certain radius 
# modified from here: http://hadoopguru.blogspot.com/2014/12/python-generate-random-latitude-and.html
# changed to take in an empty initial dataframe and load in the data + return it
# this calls the reverse geocode function to grab the addresses of each randomly generated lat & lng

def generate_latlng(latitude, longitude, df):
    
    radius = 10000                         #Choose your own radius
    radiusInDegrees=radius/111300            
    r = radiusInDegrees

    counter = 0
    
    for i in range(1,200):                 #Choose number of Lat Long to be generated

        u = float(random.uniform(0.0,1.0))
        v = float(random.uniform(0.0,1.0))

        w = r * math.sqrt(u)
        t = 2 * math.pi * v
        x = w * math.cos(t) 
        y = w * math.sin(t)

        xLat  = x + latitude
        yLng = y + longitude

        df.set_value(counter, "latitude", xLat)
        df.set_value(counter, "longitude", yLng)
        
        #print(format(counter) + ": " + format(xLat) + ", " + format(yLng))
        address = reverse_geocode(xLat, yLng).split(',')
        citystatezip = address[1] + address[2]
        
        df.set_value(counter, "address", address[0])
        df.set_value(counter, "city_state_zip", citystatezip)
        
        # Add to counter
        counter = counter + 1
    
    return df

# use Los Angeles as the center of the Latitude & longitude to test this out
# if instructor allows, we should probably just manually call this 20 times to get data for each city faster
# otherwise the gigantic loop will take forever

la_df = pd.DataFrame({"latitude":'',
                   "longitude":'',
                   "address":'',
                   "city_state_zip":''}, index=[0])

generate_latlng(34.0522342,-118.2436849, la_df)

,address,city_state_zip,latitude,longitude
0,2000 Alameda St,Vernon CA 90058,34.0172,-118.239
1,943 S Duncan Ave,East Los Angeles CA 90022,34.022,-118.171
2,799 Towne Ave,Los Angeles CA 90021,34.0378,-118.247
3,3300-3314 Idell St,Los Angeles CA 90065,34.0884,-118.222
4,1736 6th Ave,Los Angeles CA 90018,34.0408,-118.323
5,1046 S Broadway,Los Angeles CA 90015,34.0401,-118.258
6,4700 Bandini Blvd,Vernon CA 90058,33.9985,-118.176
7,928 E 48th St,Los Angeles CA 90011,34,-118.259
8,1223 S Record Ave,Los Angeles CA 90023,34.0182,-118.183
9,3903 Randolph St,Huntington Park CA 90255,33.9828,-118.195


In [30]:
la_df['message_code'] = ''
la_df.head()

,address,city_state_zip,latitude,longitude,message_code,message
0,1698 E 66th St,Los Angeles CA 90001,33.9795,-118.243,,508
1,6815a Pacific Blvd,Huntington Park CA 90255,33.9773,-118.226,,508
2,3128 Sunset Blvd,Los Angeles CA 90026,34.085,-118.275,,508
3,3457 Ellison St,Los Angeles CA 90063,34.0574,-118.189,,0
4,4900 Almaden Dr,Los Angeles CA 90042,34.1249,-118.206,,508


In [ ]:
### UNFINISHED CODE FROM THIS POINT ON ###
### STILL BEING WORKED ON ###

# TO-DO: handle error code where house does not exist at that address, skip over and/or cull from table

# zillow url format
# http://www.zillow.com/webservice/GetSearchResults.htm?zws-id=<ZWSID>&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA
        
# this shit does not work right now

for index, row in la_df.iterrows():

    try:
        url = 'https://www.zillow.com/webservice/GetSearchResults.htm?zws-id='
        address = row['address']
        citystatezip =row['city_state_zip']


        query_url = url + zws_id + '&address=' + urllib.parse.quote(address) + '&citystatezip=' + urllib.parse.quote(citystatezip) 
        #print(query_url)

        # import time, put time.sleep in between each call ??
        root = ET.parse(urlopen(query_url)).getroot()

        for message in root.iter('message'):
            message_code = message[1].text
        
        print(format(index) + ": " + message_code)
        
        la_df.set_value(index, 'message_code', message_code)
    
    except:
        break
        

    

0: 7
1: 7
2: 7
3: 7
4: 508
5: 7
6: 7
7: 7
8: 7
9: 7
10: 7
11: 7
12: 7
13: 7
14: 7
15: 7
16: 7
17: 0
18: 7
19: 7
20: 7
21: 7
22: 7
23: 7
24: 7
25: 7
26: 7
27: 7
28: 7
29: 7
30: 7
31: 7
32: 7
33: 7
34: 7
35: 7
36: 7
37: 7
38: 7
39: 7
40: 7
41: 7
42: 7
43: 7
44: 7
45: 7
46: 7
47: 508
48: 7
49: 7
50: 7
51: 7
52: 7
53: 7
54: 7
55: 7
56: 7
57: 7
58: 7
59: 7
60: 7
61: 7
62: 7
63: 7
64: 7


In [ ]:
la_df

# traverse xml to pull certain pieces of data out


    

In [38]:
# traverse xml to pull certain pieces of data out

root = ET.parse(urllib.request.urlopen(query_url)).getroot()

# grab the zestimate
for zestimate in root.iter('zestimate'):
    print (zestimate[0].text)

# grab the difference in valuation
for valuation in root.iter('valuationRange'):
    lowValuation = int(valuation[0].text)
    highValuation = int(valuation[1].text)
    valuationRange = highValuation - lowValuation
    print(valuationRange)
    #print(lowValuation)
    #print(highValuation)

# grab the following
# address
# price 
# valuation range

822934
82294
